In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout, LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
#Create a path to where the data is stored.
path = 'C:/Users/kaan/ML_2'

In [3]:
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'raw.csv'))

In [4]:
#import pleasant weather data
df_pw = pd.read_csv(os.path.join(path,'02 Data','Original data','Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [5]:
# preparing dataset for deep learning
df.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.018,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.018,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.018,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [6]:
df_pw.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
# dropping 'DATE' and 'MONTH' columns from observations data set (df)
df = df.drop(columns=['DATE', 'MONTH'], axis=1)
df.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,6.1,3.3,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.30,0,0.0,8.5,5.1,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,0.00,0,4.1,6.3,3.8,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,0.14,0,5.4,3.0,-0.7,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [8]:
# dropping 'DATE' column from predictions data set (df_pw)
df_pw = df_pw.drop(columns=['DATE'], axis=1)
df_pw.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# Dropping all 'GDANSK', 'ROMA','TOURS' columns from observations dataset(df) because there is no related data to them in pleasent weather dataset
cols = [col for col in df.columns if col.startswith(('GDANSK', 'ROMA', 'TOURS'))]
df = df.drop(columns=cols)

In [11]:
df.columns

Index(['BASEL_cloud_cover', 'BASEL_wind_speed', 'BASEL_humidity',
       'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation',
       'BASEL_snow_depth', 'BASEL_sunshine', 'BASEL_temp_mean',
       'BASEL_temp_min',
       ...
       'VALENTIA_cloud_cover', 'VALENTIA_humidity', 'VALENTIA_pressure',
       'VALENTIA_global_radiation', 'VALENTIA_precipitation',
       'VALENTIA_snow_depth', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
       'VALENTIA_temp_min', 'VALENTIA_temp_max'],
      dtype='object', length=147)

In [16]:
# Finding missing values in columns and station based missing values.
# Get all observation types (part after the first "_")
observation_types = sorted({col.split('_', 1)[1] for col in df.columns})

print("Number of stations for each observation type:\n")
for obs in observation_types:
    # Columns that belong to this observation type
    cols_for_obs = [c for c in df.columns if c.endswith(obs)]
    # Number of stations = number of columns with this observation type
    print(f"{obs:15s}: {len(cols_for_obs)} stations")

Number of stations for each observation type:

cloud_cover    : 14 stations
global_radiation: 15 stations
humidity       : 14 stations
precipitation  : 15 stations
pressure       : 14 stations
snow_depth     : 6 stations
sunshine       : 15 stations
temp_max       : 15 stations
temp_mean      : 15 stations
temp_min       : 15 stations
wind_speed     : 9 stations


### wind_speed and snow_depth have so many missing stations value that's why these columns will be dropped.

In [18]:
# All stations (part before the first "_")
stations = sorted({col.split('_')[0] for col in df.columns})

# Choose which observation types you want to check
obs_to_check = ['cloud_cover', 'humidity', 'pressure']  # since these are the ones have 14 stations values

print("\nStations missing each observation type:\n")
for obs in obs_to_check:
    # Columns that belong to this observation type
    cols_for_obs = [c for c in df.columns if c.endswith(obs)]
    stations_with_obs = {c.split('_')[0] for c in cols_for_obs}

    # Stations that exist but do not have this observation type at all
    missing_stations = sorted(set(stations) - stations_with_obs)

    print(f"{obs}:")
    if missing_stations:
        for s in missing_stations:
            print("  ", s)
    else:
        print("  none")
    print()


Stations missing each observation type:

cloud_cover:
   KASSEL

humidity:
   STOCKHOLM

pressure:
   MUNCHENB



### Values will be inserted these missing stations. The values is taken from their neighbour stations.

In [19]:
#dropping columns that has less value
cols_to_drop = [col for col in df.columns
    if col.endswith("wind_speed") or col.endswith("snow_depth")]

df = df.drop(columns=cols_to_drop)

In [20]:
df.shape

(22950, 132)

In [24]:
# filling the values to the stations
fill_plan = [
    ("KASSEL",    "cloud_cover", "LJUBLJANA"),
    ("STOCKHOLM", "humidity",    "OSLO"),
    ("MUNCHENB",  "pressure",    "SONNBLICK")  
]

for target_station, obs, donor_station in fill_plan:
    target_col = f"{target_station}_{obs}"
    donor_col  = f"{donor_station}_{obs}"

    # If target column already exists: fill only NaN values
    if target_col in df.columns:
        df[target_col] = df[target_col].fillna(df[donor_col])
    # If target column does not exist: create it by copying donor column
    else:
        df[target_col] = df[donor_col].values

In [25]:
df.shape

(22950, 135)

In [26]:
df_pw.shape

(22950, 15)

In [27]:
#exporting cleaned datasets
df.to_csv(os.path.join(path,'02 Data','Prepared Data', 'weather_cleaned.csv'), index=False)
df_pw.to_csv(os.path.join(path,'02 Data','Prepared Data', 'predictions_cleaned.csv'), index=False)

# Data Reshape

In [28]:
# Renaming datasets
X=df
y=df_pw

In [29]:
X.shape

(22950, 135)

In [30]:
y.shape

(22950, 15)

In [31]:
# Turn X and y into arrays
X = np.array(X)
y = np.array(y)

In [32]:
X = X.reshape(-1,15,9)

In [33]:
X.shape

(22950, 15, 9)

In [34]:
X

array([[[ 7.0000e+00,  8.5000e-01,  1.0180e+00, ...,  6.5000e+00,
          8.0000e-01,  1.0900e+01],
        [ 1.0000e+00,  8.1000e-01,  1.0195e+00, ...,  3.7000e+00,
         -9.0000e-01,  7.9000e+00],
        [ 4.0000e+00,  6.7000e-01,  1.0170e+00, ...,  2.4000e+00,
         -4.0000e-01,  5.1000e+00],
        ...,
        [ 1.0304e+00,  4.8000e-01,  1.0000e-02, ..., -3.2000e+00,
          5.0000e+00,  1.0114e+00],
        [ 5.0000e-02,  3.2000e-01,  0.0000e+00, ...,  5.0000e+00,
          8.8000e-01,  1.0003e+00],
        [ 4.5000e-01,  3.4000e-01,  4.7000e+00, ...,  8.0000e+00,
          9.8000e-01,  1.0304e+00]],

       [[ 6.0000e+00,  8.4000e-01,  1.0180e+00, ...,  6.1000e+00,
          3.3000e+00,  1.0100e+01],
        [ 6.0000e+00,  8.4000e-01,  1.0172e+00, ...,  2.9000e+00,
          2.2000e+00,  4.4000e+00],
        [ 4.0000e+00,  6.7000e-01,  1.0170e+00, ...,  2.3000e+00,
          1.4000e+00,  3.1000e+00],
        ...,
        [ 1.0292e+00,  2.1000e-01,  6.1000e-01, ..., -

# Splitting the data

In [35]:
# Splitting data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

# Keras model

In [36]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu
model.summary()

C:\Users\kaan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 14, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,831 (11.06 KB)

 Trainable params: 2,831 (11.06 KB)

 Non-trainable params: 0 (0.00 B)

# Compile and running the model

In [37]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 3s - 2ms/step - accuracy: 0.1107 - loss: 6654.5757
Epoch 2/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1202 - loss: 74921.3516
Epoch 3/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1174 - loss: 235141.2656
Epoch 4/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1168 - loss: 499623.4688
Epoch 5/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1193 - loss: 919783.8750
Epoch 6/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1203 - loss: 1412516.8750
Epoch 7/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1186 - loss: 2035650.1250
Epoch 8/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1221 - loss: 2805694.0000
Epoch 9/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1222 - loss: 3791503.5000
Epoch 10/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1222 - loss: 4836727.5000
Epoch 11/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1209 - loss: 6084226.0000
Epoch 12/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1198 - loss: 7571998.0000
Epoch 13/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1216 - loss: 9160226.0000
Epo

# Confusion Matrix

In [39]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [40]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Pred        BASEL  BELGRADE  DEBILT  DUSSELDORF  HEATHROW  KASSEL  LJUBLJANA  \
True                                                                           
BASEL         159        32     564         187         1       1       2626   
BELGRADE        0         0     110           0         0       0        982   
BUDAPEST        0         0      41           0         0       0        173   
DEBILT          0         0      32           0         0       0         50   
DUSSELDORF      0         0       9           0         0       0         20   
HEATHROW        0         0      29           0         0       0         53   
KASSEL          0         0       5           0         0       0          6   
LJUBLJANA       0         0       4           0         0       0         57   
MAASTRICHT      0         0       2           0         0       0          7   
MADRID         10         0      83           5         0       0        356 

In [41]:
model.evaluate(X_test, y_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0434 - loss: 75456032.0000   


[75456032.0, 0.04339491203427315]

# Model improvement steps

In [42]:
epochs = 30
batch_size = 32
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu
model.summary()

C:\Users\kaan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
538/538 - 2s - 3ms/step - accuracy: 0.1130 - loss: 2134.1902
Epoch 2/30
538/538 - 1s - 2ms/step - accuracy: 0.1183 - loss: 20083.2402
Epoch 3/30
538/538 - 1s - 2ms/step - accuracy: 0.1200 - loss: 70001.1094
Epoch 4/30
538/538 - 1s - 2ms/step - accuracy: 0.1189 - loss: 162954.9531
Epoch 5/30
538/538 - 1s - 2ms/step - accuracy: 0.1221 - loss: 302777.8750
Epoch 6/30
538/538 - 1s - 2ms/step - accuracy: 0.1174 - loss: 481251.2500
Epoch 7/30
538/538 - 1s - 2ms/step - accuracy: 0.1199 - loss: 709898.1250
Epoch 8/30
538/538 - 1s - 2ms/step - accuracy: 0.1165 - loss: 970277.8750
Epoch 9/30
538/538 - 1s - 2ms/step - accuracy: 0.1185 - loss: 1307020.0000
Epoch 10/30
538/538 - 1s - 2ms/step - accuracy: 0.1223 - loss: 1700445.1250
Epoch 11/30
538/538 - 1s - 2ms/step - accuracy: 0.1209 - loss: 2111959.5000
Epoch 12/30
538/538 - 1s - 2ms/step - accuracy: 0.1235 - loss: 2585316.7500
Epoch 13/30
538/538 - 1s - 2ms/step - accuracy: 0.1182 - loss: 3090268.0000
Epoch 14/30
538/538 - 1s - 2ms/st

In [45]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))
model.evaluate(X_test, y_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL          47       602        15     109           8         2    1595   
BELGRADE        0       272         0       2           0         0     627   
BUDAPEST        0        32         0       2           0         0     121   
DEBILT          0         0         0       1           0         0      74   
DUSSELDORF      0         2         0       1           0         0      19   
HEATHROW        0         3         0       3           0         0      49   
KASSEL          0         3         0       0           0         0       6   
LJUBLJANA       0        17         0       0           0         0      16   
MAASTRICHT      0         2         0       0           0         0       7   
MADRID          0        46         0      19           0         0     114   
MUNCHENB 

[18470522.0, 0.10230045020580292]